# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계3 : 응급상황 연계(추천)**

## **0.미션**

단계 3에서는, 응급상황의 음성을 인식해서 텍스트로 변환하고, 변환된 텍스트를 다시 요약 및 핵심키워드 도출 작업을 수행합니다.  
이를 위해 사전학습된 모델을 API로 연결하여 활용합니다.

### 미션4 : 응급실 추천
* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기


## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
path = '/content/drive/MyDrive/project6_2/'

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [15]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [16]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine,Unit
import requests
import json

# 더 필요한 라이브러리 추가 -------------

### (3) 데이터 로딩
* 단계1에서 수집한 응급실 정보를 불러와서 데이터프레임으로 저장합니다.

In [17]:
df1= pd.read_csv(path + 'df_emergency.csv')

df1.head()

,Hospital,City,District,Village,Phone_Number,Longitude,Latitude,Hospital_Type,Medical_Type,Zip_Code,Region,Address
0,가톨릭대학교성빈센트병원,경기,수원팔달구,지동,031-1577-8588,127.027427,37.277985,상급종합병원,지역응급의료센터,16247,경기도,경기도 수원시 팔달구 중부대로 93 (지동)
1,가톨릭대학교인천성모병원,인천,인천부평구,부평동,032-1544-9004,126.724899,37.484831,상급종합병원,지역응급의료센터,21431,인천광역시,인천광역시 부평구 동수로 56 (부평동)
2,강릉아산병원,강원,강릉시,사천면,033-610-3114,128.857841,37.818433,상급종합병원,권역응급의료센터,25440,강원특별자치도,강원특별자치도 강릉시 사천면 방동길 38
3,강북삼성병원,서울,종로구,평동,02-2001-2001,126.967750,37.568408,상급종합병원,지역응급의료센터,3181,서울특별시,서울특별시 종로구 새문안로 29 (평동)
4,건국대학교병원,서울,광진구,화양동,1588-1533,127.071828,37.540376,상급종합병원,지역응급의료센터,5030,서울특별시,서울특별시 광진구 능동로 120-1 (화양동)


## **2. 응급실 추천**


### (1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
- 세부사항
    * 하버사인 함수를 활용


#### 1) 하버사인 함수 사용 연습
* 임의의 두 좌표간 거리 계산
    * 응급실 데이터프레임을 열어서
    * 응급실 두 곳의 좌표를 확인하고
    * 두 지점의 거리를 계산해 봅시다
* 사용법 : haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [18]:
coords_1 = (df1.loc[0, "Latitude"], df1.loc[0, "Longitude"])
coords_2 = (df1.loc[1, "Latitude"], df1.loc[1, "Longitude"])

# 응급실 간 거리 계산

distance = haversine(coords_1, coords_2, unit='km')

print(f"응급실 간 거리: {distance:.2f} km")


# 48km

응급실 간 거리: 35.26 km


#### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후
    * 가장 가까운 거리의 응급실 3 곳을 반환합니다.
* 이를 함수로 생성하고 테스트 해 봅시다.

In [19]:
def closest_emergency_rooms(patient_coords, emergency_rooms_df, top=3):

    # 응급실 좌표와 입력 좌표 간의 거리 계산
    emergency_rooms_df['거리(km)'] = emergency_rooms_df.apply(
        lambda row: haversine(patient_coords, (row['Latitude'], row['Longitude']), unit='km'), axis=1)

    # 거리가 짧은 순으로 정렬하고, top개의 응급실 반환
    closest_rooms = emergency_rooms_df[['Hospital', '거리(km)']].sort_values(by='거리(km)').head(top)

    return closest_rooms

# 환자 위치 (위도, 경도)
patient_coords = (35.10380669389722, 126.87503942538945)

# 함수 실행
closest_rooms = closest_emergency_rooms(patient_coords, df1)

In [20]:
closest_rooms

,Hospital,거리(km)
153,빛고을전남대학교병원,2.199595
89,광주씨티병원,2.708522
160,서광병원,3.651228


#### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율 적입니다.
* 해결 방안 : 그래서 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정 합니다.
* hint :
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고
    * 응급실 정보에서 해당 구간을 먼저 조회한 후
    * 거리 계산

In [26]:
# ±α 일정 범위로 가까운 응급실 3곳 추천하기

def recommend_hospital2(patient_coords, emergency_rooms_df, top=3, a=1.0):

    # 환자 위치 -> 구간 계산

    # 위도 ±α
    lat_min = patient_coords[0] - a
    lat_max = patient_coords[0] + a
    # 경도 ±α
    lon_min = patient_coords[1] - a
    lon_max = patient_coords[1] + a

    # α 범위 내의 응급실만 추출
    filtered_rooms = emergency_rooms_df[
        (emergency_rooms_df['Latitude'] >= lat_min) & (emergency_rooms_df['Latitude'] <= lat_max) &
        (emergency_rooms_df['Longitude'] >= lon_min) & (emergency_rooms_df['Longitude'] <= lon_max)
    ]

    # 추출된 응급실과 거리 계산
    filtered_rooms['거리(km)'] = filtered_rooms.apply(
        lambda row: haversine(patient_coords, (row['Latitude'], row['Longitude']), unit='km'), axis=1)

    # 거리 기준으로 정렬하고, top개의 응급실 반환
    closest_rooms = filtered_rooms[['Hospital', '거리(km)']].sort_values(by='거리(km)').head(top)

    return closest_rooms


#  환자 위치 (위도, 경도)
patient_coords = (35.10380669389722, 126.87503942538945)

# 함수 실행
closest_rooms_2= recommend_hospital2(patient_coords, df1, top=3, a=1.0)

<ipython-input-26-f0222c7890aa>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rooms['거리(km)'] = filtered_rooms.apply(


In [22]:
closest_rooms_2

,Hospital,거리(km)
153,빛고을전남대학교병원,2.199595
89,광주씨티병원,2.708522
160,서광병원,3.651228


### (2) [조 과제]고도화 : naver 지도 api 사용

* 이 부분은 조별 과제로 수행하게 됩니다.(개인과제 아님!)

* 세부사항
    * 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수를 생성하시오.
    * 함수 내용
        * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
        * 출력 : 도로거리(km)
    
    * 네이버 Maps API 활용
        * 사용할 API : Direction 5
        * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
        * 가이드를 활용해서 url, header, params를 구성합니다.
        * params의 옵션은 'trafast' (실시간 빠른 길 옵션)을 선택하시오.

#### 1) maps 클라이언트ID, 키 로딩

In [23]:
c_id = 'hgs1dklcmi'
c_key = 'na8wNh9hmCkOzxOmuCR2d3rMeULallJa4p75zWP0'

#### 2) 함수 생성

In [24]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)

    # 응답 상태 코드 확인
    if response.status_code == 200:
        data = response.json()
        # 실시간 빠른 길의 경로 정보에서 거리 정보 추출
        dist = data['route']['trafast'][0]['summary']['distance']  # 거리(m)
        # 반환: 거리(km)
        return dist / 1000  # km 단위로 변환하여 반환
    else:
        return f"Error: {response.status_code}, {response.text}"


c_id = 'hgs1dklcmi'
c_key = 'na8wNh9hmCkOzxOmuCR2d3rMeULallJa4p75zWP0'
start_lat = 37.3597080  # 출발지 위도
start_lng = 127.1058342  # 출발지 경도
dest_lat = 35.179470  # 목적지 위도
dest_lng = 129.075986  # 목적지 경도

distance = get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key)

* 테스트

In [25]:
print(f"도로 거리: {distance} km")

도로 거리: 387.553 km


#### 3) 응급실 추천
* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [29]:
import requests

# get_dist 함수 (네이버 지도 API를 사용하여 도로 거리 계산)
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, headers=headers, params=params)

    # 응답 상태 코드 확인
    if response.status_code == 200:
        data = response.json()
        # 실시간 빠른 길의 경로 정보에서 거리 정보 추출
        dist = data['route']['trafast'][0]['summary']['distance']  # 거리(m)
        # 반환: 거리(km)
        return dist / 1000  # km 단위로 변환하여 반환
    else:
        return f"Error: {response.status_code}, {response.text}"

In [44]:
def recommend_hospital3(patient_coords, emergency_rooms_df, c_id, c_key, top=3, initial_a=0.1, increment=0.1):

    a = initial_a  # 초기 범위 설정

    while True:  # 후보가 충분할 때까지 반복
        lat_min = patient_coords[0] - a
        lat_max = patient_coords[0] + a
        lon_min = patient_coords[1] - a
        lon_max = patient_coords[1] + a

        # α 범위 내의 응급실만 추출
        filtered_rooms = emergency_rooms_df[
            (emergency_rooms_df['Latitude'] >= lat_min) & (emergency_rooms_df['Latitude'] <= lat_max) &
            (emergency_rooms_df['Longitude'] >= lon_min) & (emergency_rooms_df['Longitude'] <= lon_max)
        ]

        # 후보가 충분하면 거리 계산
        if len(filtered_rooms) > 0:
            filtered_rooms['거리(km)'] = filtered_rooms.apply(
                lambda row: get_dist(patient_coords[0], patient_coords[1], row['Latitude'], row['Longitude'], c_id, c_key), axis=1
            )

            # 거리 기준으로 정렬하고, top개의 응급실 반환
            closest_rooms = filtered_rooms[['Hospital', '거리(km)']].sort_values(by='거리(km)').head(top)
            return closest_rooms

        # 후보가 부족하면 범위를 늘림
        a += increment

        # 무한 루프 방지
        if a > 5.0:
            return pd.DataFrame({'Hospital': [], '거리(km)': []})

# 네이버 지도 API 클라이언트 ID와 키
c_id = 'hgs1dklcmi'
c_key = 'na8wNh9hmCkOzxOmuCR2d3rMeULallJa4p75zWP0'

# 환자 위치 (위도, 경도)
patient_coords = (35.10380669389722, 126.87503942538945)

# 함수 실행
closest_rooms_3 = recommend_hospital3(patient_coords, df1, c_id, c_key, top=3, initial_a=0.1)

<ipython-input-44-ac2efa5c987f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rooms['거리(km)'] = filtered_rooms.apply(


In [45]:
closest_rooms_3

,Hospital,거리(km)
89,광주씨티병원,3.336
153,빛고을전남대학교병원,4.540
160,서광병원,4.598


In [42]:
# def recommend_hospital3(patient_coords, emergency_rooms_df, c_id, c_key, top=3, a=0.1):
#     # 환자 위치 -> 구간 계산
#     lat_min = patient_coords[0] - a
#     lat_max = patient_coords[0] + a
#     lon_min = patient_coords[1] - a
#     lon_max = patient_coords[1] + a

#     # α 범위 내의 응급실만 추출
#     filtered_rooms = emergency_rooms_df[
#         (emergency_rooms_df['Latitude'] >= lat_min) & (emergency_rooms_df['Latitude'] <= lat_max) &
#         (emergency_rooms_df['Longitude'] >= lon_min) & (emergency_rooms_df['Longitude'] <= lon_max)
#     ]

#     # 응급실과 거리 계산
#     filtered_rooms['거리(km)'] = filtered_rooms.apply(
#         lambda row: get_dist(patient_coords[0], patient_coords[1], row['Latitude'], row['Longitude'], c_id, c_key), axis=1)

#     # 거리 기준으로 정렬하고, top개의 응급실 반환
#     closest_rooms = filtered_rooms[['Hospital', '거리(km)']].sort_values(by='거리(km)').head(top)

#     return closest_rooms

# # 네이버 지도 API 클라이언트 ID와 키
# c_id = 'hgs1dklcmi'
# c_key = 'na8wNh9hmCkOzxOmuCR2d3rMeULallJa4p75zWP0'

# # 환자 위치 (위도, 경도)
# patient_coords = (35.10380669389722, 126.87503942538945)

# # 함수 실행
# closest_rooms_3 = recommend_hospital3(patient_coords, df1, c_id, c_key, top=3, a=1.0)

<ipython-input-42-7f6b6f39ab7a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rooms['거리(km)'] = filtered_rooms.apply(


In [43]:
# closest_rooms_3

,Hospital,거리(km)
89,광주씨티병원,3.336
153,빛고을전남대학교병원,4.540
160,서광병원,4.598


## **Mission Complete!**

수고 많았습니다!